In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
#!pip install transformers datasets accelerate gradio

In [3]:
import torch
print(torch.__version__)

2.5.1+cu121


In [3]:
import os
import numpy as np
import pandas as pd
import torch
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)

from datasets import Dataset

C:\Users\nithi\anaconda3\envs\my_ml_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.environ["WANDB_DISABLED"] = "true"

In [5]:
df = pd.read_csv(r"C:/Users/nithi/Desktop/Nithin/Nicholas/train.csv")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
TEXT_COL = "comment_text"

LABELS = [
    "toxic",
    "severe_toxic",
    "obscene",
    "threat",
    "insult",
    "identity_hate"
]

In [8]:
dataset = Dataset.from_pandas(df)

In [9]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [10]:
def tokenize(batch):
    return tokenizer(batch[TEXT_COL], padding=True, truncation=True, max_length=256)
enc = dataset.map(tokenize, batched=True)

Map: 100%|████████████████████████████████████████████████████████████| 159571/159571 [00:41<00:00, 3860.55 examples/s]


In [11]:
def format_labels(example):
    example["labels"] = [float(example[c]) for c in LABELS]
    return example

enc = enc.map(format_labels)

Map: 100%|████████████████████████████████████████████████████████████| 159571/159571 [00:55<00:00, 2856.56 examples/s]


In [12]:
enc = enc.train_test_split(test_size=0.2)

test_split = enc["test"]
train_split = enc["train"].train_test_split(test_size=0.1)

train_enc = train_split["train"]
val_enc   = train_split["test"]
test_enc  = test_split

In [13]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(LABELS),
    problem_type="multi_label_classification",
    from_tf=False
)

model.to("cuda")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("GPU training device =", device)

GPU training device = cuda


In [15]:
training_args = TrainingArguments(
    output_dir="./bert_checkpoints",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=val_enc,
    tokenizer=tokenizer
)

trainer.model.to(device)  

C:\Users\nithi\AppData\Local\Temp\ipykernel_3140\1303073767.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [19]:
trainer.train()

Step,Training Loss
500,0.109500
1000,0.050500
1500,0.048800
2000,0.046200
2500,0.046200
3000,0.045800
3500,0.041100
4000,0.044300
4500,0.043200
5000,0.040600


TrainOutput(global_step=14362, training_loss=0.03994944108560081, metrics={'train_runtime': 3340.5812, 'train_samples_per_second': 68.784, 'train_steps_per_second': 4.299, 'total_flos': 1.522026508142592e+16, 'train_loss': 0.03994944108560081, 'epoch': 2.0})

In [20]:
trainer.evaluate()

{'eval_loss': 0.03531823679804802,
 'eval_runtime': 62.9886,
 'eval_samples_per_second': 202.672,
 'eval_steps_per_second': 12.669,
 'epoch': 2.0}

In [21]:
def predict_comment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    logits = model(**inputs).logits
    probs = torch.sigmoid(logits).detach().cpu().numpy()[0]
    pred = (probs > 0.5).astype(int)

    return dict(zip(LABELS, pred))

In [25]:
predict_comment("I respect your opinion.")

{'toxic': 0,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 0,
 'identity_hate': 0}

In [26]:
predict_comment("You are a disgrace and a waste of space.")

{'toxic': 1,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 1,
 'identity_hate': 0}

In [27]:
import gradio as gr

def ui_fn(comment):
    result = predict_comment(comment)
    return "\n".join([f"{k}: {bool(v)}" for k,v in result.items()])

ui = gr.Interface(
    fn=ui_fn,
    inputs=gr.Textbox(lines=3, placeholder="Enter a comment"),
    outputs="text"
)

ui.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [28]:
from transformers import AutoTokenizer

# Save model + tokenizer
model.save_pretrained("./bert_toxicity_model")
tokenizer.save_pretrained("./bert_toxicity_model")

print("Model saved successfully!")

Model saved successfully!
